<a href="https://colab.research.google.com/github/eric052/PIS2022/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage,ConfirmTemplate,MessageAction,
    URIAction,
    CarouselTemplate,
    CarouselColumn,
    PostbackAction
)


from linebot.v3.messaging import TemplateMessage, ConfirmTemplate, MessageAction


app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))




@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
          reply = TemplateMessage(
            alt_text='這是一個確認視窗',
            template=ConfirmTemplate(
              text="你確定要進行這個動作嗎？",
              actions=[
                  MessageAction(label="是", text="是的"),
                  MessageAction(label="否", text="否的")
              ]
            )
          )
        elif action =='carousel':
            carousel_template = CarouselTemplate(
              columns=[
                CarouselColumn(
                    thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                    title='夏日大作戰',
                    text='細田守導演的日本科幻暨浪漫電影',
                    actions=[
                        URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),
                        MessageAction(label='投票', text='投夏日大作戰一票')
                    ]
                ),
                CarouselColumn(
                  thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/4/4f/Castle_of_Cagliostro_poster.png',
                  title='魯邦三世 卡里奧斯特羅城',
                  text='宮崎駿導演的日本動畫動作冒險喜劇電影。',
                  actions=[
                    URIAction(label='維基百科', uri='https://example.com/productB'),
                    MessageAction(label='投票', text='魯邦三世-卡里奧斯特羅城一票')
                  ]
                )
              ]
            )
            reply = TemplateMessage(
              alt_text='這是一個輪播模板',
              template=carousel_template
            )
        else:
          reply = TextMessage(text='Thanks!')

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                  reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9223-35-229-233-251.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:10:59] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:11:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:11:27] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:11:49] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:11:56] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:12:12] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:12:15] "POST /callback HTTP/1.1" 200 -
